- 1단계, 상자로서의 변수
- 2단계, 변수를 낳는 함수
- 3단계, 함수 연결
- 4단계, 수치 미분
- 5단계, 역전파 이론
- 6단계, 수동 역전파
- 7단계, 역전파 자동화
- 8단계, 재귀에서 반복문으로
- 9단계, 함수를 더 편리하게
- 10단계, 테스트



## 1단계, 상자로서의 변수

### Variable 클래스 구현

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


### numpy의 다차원배열

In [3]:
import numpy as np
x = np.array(1)
x.ndim

0

In [4]:
x = np.array([1, 2, 3])
x.ndim

1

In [5]:
x = np.array([1,2,3], [4,5,6])
x.ndim

TypeError: Field elements must be 2- or 3-tuples, got '4'

In [6]:
x = np.array([[1,2,3], [4,5,6]])
x.ndim

2

## 2단계, 변수를 낳는 함수

### Function 클래스 구현

In [8]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2
        output = Variable(y)
        return output

### Function 클래스 이용

In [10]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [11]:
type(y)

__main__.Variable

In [12]:
y.data

100

- Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
- 구체적인 함수는 Function클래스를 상속한 클래스에서 구현합니다

In [15]:
class Function:
    def __call__(self, input):
        x = input.data
        # 구체적인 계산은 forward메서드에서 한다.
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        ## 이메서드는 상속하여 구현해야함을 명시해줌.
        raise NotImplementError()

In [16]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [17]:
x = Variable(np.array(10))
f = Square()
y = f(x)

In [18]:
type(y)

__main__.Variable

In [19]:
y.data

100

### 3단계, 함수 연결

In [21]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [22]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
y.data

1.648721270700128

#### 위의 식을 그림으로 그려서 이해해보기

### 4단계, 수치미분

- 컴퓨터는 극한을 취급할 수 없으니, h를 극한과 비슷한 값으로 대체. h= 0.0001과 같은 매우 작은 값을 이용하여 식(f'(x)= lim (f(x+h)-f(x))/h) 을 계산합니다 이런 미세한 차이를 이용하여 함수의 변화량을 구하는 방법을 수치 미분(numerical differentiation) 이라고 함.

In [24]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [26]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
dy

4.000000000004

#### y = x^2, x = 2.0

### 합성 함수의 미분

In [27]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

In [28]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
dy

3.2974426293330694

### 수치 미분의 문제점
- 수치 미분의 결과에는 오차가 포함되어 있음.
- 수치 미분의 결과에 오차가 포함되기 쉬운 이유는 주로 '자릿수 누락' 때문.
- 수치 미분의 더 심각한 문제는 계산량이 많다는 점. 그래서 등장한 것이 바로 역전파.
- 수치 미분은 구현하기 쉽고, 거의 정확한 값을 얻을 수 있다. 이에 비해 역전파는 복잡한 알고리즘이라서 구현하면서 버그가 섞여 들어가기 쉽다. 그래서 역전파를 정확하게 구현했는지 확인하기 위해 수치 미분의 결과를 이용하곤한다. 이를 기울기 확인(gradient checking). 단순히 수치 미분 결과와 역전파의 결과를 비교하는 것.

## 5단계, 역전파 이론

- 역전파를 이용하면 미분을 효율적으로 계산할 수 있고, 결과값의 오차도 더 작다.

### 연쇄 법칙

### 역전파 원리 도출

### 계산 그래프로 살펴보기

## 6단계, 수동 역전파

### Variable 클래스 추가

In [36]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

### Function 클래스 추가 구현

- 미분을 계산하는 역전파(backward 메서드)
- forward 메섣 호출 시 건네받은 Variable 인스턴스 유지

In [37]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):
        raise NotImplementedError()

In [54]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [55]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
    

### 역전파 구현

In [40]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [41]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
x.grad

3.297442541400256

## 7단계, 역전파 자동화

### 역전파 자동화의 시작

In [52]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    def set_creator(self, func):
        self.creator = func

In [53]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

- 주의 Function 클래스를 재정의했으므로  Square클래스와 Exp 클래스 정의를 재호출해야한다.

In [56]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라간다.
### Defined by Run
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x


### 역전파 도전

In [58]:
y.grad = np.array(1.0)

C = y.creator                 # 1. 함수를 가져온다.
b = C.input                   # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad)   # 3. 함수의 backward 메서드를 호출한다.

In [59]:
B = b.creator                 # 1. 함수를 가져온다.
a = B.input                   # 2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad)   # 3. 함수의 backward 메서드를 호출한다.

In [60]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
x.grad

3.297442541400256

### backward 메서드 추가

In [61]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator        # 1. 함수를 가져온다.
        if f is not None:
            x = f.input                       # 2. 함수의 입력을 가져온다.
            x .grad = f.backward(self.grad)   # 3. 함수의 backward메서드를 호출한다.
            x.backward()                      # 하나 앞 변수의 backward 메서드를 호출한다.(재귀)

In [62]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)


3.297442541400256
